<a href="https://colab.research.google.com/github/kscaman/MAP583/blob/main/TP/TP03_optimization_algorithms.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Optimization algorithms

Code adapted from the [optimization chapter](http://www.d2l.ai/chapter_optimization/) of Dive into Deep Learning. The objectives are:

1.   To show the advantages and disadvantages of several popular optimization algorithms on simple 2d functions.
2.   To explore the impact of gradient noise and mini-batch size on the training of simple models.


We will thus fix the function to learn, and try several optimization algorithms on it. The following functions are used to access the gradient (`get_gradient`), run an optimization loop with a given update rule (`optimize`), and show the trace of this optimization loop on a figure (`show_trace_2d`). `show_trace_2d` requires as inputs: the function to optimize and the output of the `optimize` function. The `update` rule performs one iteration of the optimization algorithm, and will use internally `get_gradient` to access and use the gradient.

In [3]:
%matplotlib inline

import numpy as np
import torch
import matplotlib.pyplot as plt


def get_gradient(f, x, noise_std=0):
    """Returns the gradient of a function f at x with additive Gaussian noise."""
    x = x.detach() # Removes information about the gradient
    x.requires_grad = True
    output = f(x)
    output.backward()
    gradient = x.grad
    noise = torch.randn(gradient.shape)
    return gradient + noise_std * noise

def optimize(update, num_iter=20, init_s=0):
    """Optimize the objective function of 2d variables with a customized update."""
    """update(x,s_x) should return the updated positions x and possible memory terms."""
    x = torch.Tensor([-5, -2])
    s = init_s
    all_x = [x]
    for i in range(num_iter):
        x, s = update(x, s)
        all_x.append((x))
    print('epoch %d, x1 %f, x2 %f' % (i+1, x[0], x[1]))
    return torch.stack(all_x, dim=0)

def show_trace_2d(f, all_x, color='red', is_new_plot=True):
    """Show the trace of 2d variables during optimization."""
    # Plot the iterates x_t
    if is_new_plot:
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))
    else:
        fig = plt.gcf()
        ax1 = fig.axes[0]
        ax2 = fig.axes[1]
    ax1.plot(all_x[:,0], all_x[:,1], '-o', color=color)
    all_f = np.array([f(torch.Tensor([x[0], x[1]])) for x in all_x])

    # Plot the function's level sets
    x1 = np.arange(-5.5, 3.5, 0.1)
    x2 = np.arange(-3.0, 2.0, 0.1)
    f_grid = np.array([[f(torch.Tensor([u, v])) for u in x1] for v in x2])
    ax1.contour(x1, x2, f_grid, colors='blue')
    ax1.set_xlim([-5.5, 3.5])
    ax1.set_ylim([-3.0, 2.0])
    ax1.set_xlabel('x1')
    ax1.set_ylabel('x2')

    # Plot the function values f(x_t)
    ax2.plot(all_f, color=color)
    ax2.set_xlabel('time ($t$)')
    ax2.set_ylabel('function value ($f(x_t)$)')

Write a function `f` that returns $f(x) = 0.1 x_0^2 + 2 x_1^2$.

In [4]:
def f(x):
    return 0.1*f[0]**2+2*f[1]**2

## Gradient descent

Write a function `gd(x, s)` that takes as input a point `x` and (unused) memory state `s`, and performs one iteration (update rule) of gradient descent. Remember that `get_gradient(f, x)` returns the gradient of `f` at `x`. Then, test your update rule with step sizes `eta` equal to $0.4$ and $0.6$ using `show_trace_2d(f, optimize(gd))`.

In [8]:
eta = 0.4

def gd(x, s):

    grad=get_gradient(f, x)

    for i in x:
        xi=x[i]
        x[i]=xi-eta*grad[i]

    return x


AttributeError: 'numpy.ndarray' object has no attribute 'detach'

## Momemtum

Write a function `momentum(x, v)` that performs one iteration of the momentum method. Test your update rule on the same step-sizes as gradient descent. What happens? Try also with `eta, gamma = 0.05, 0.9`.

In [ ]:
eta, gamma = 0.4, 0.5

#
# YOUR CODE HERE
#

## Nesterov accelerated gradient

Now write the function `nesterov(x, v)` for Nesterov's accelerated grdient descent. Compare with momentum with the same hyper-parameters `eta, gamma = 0.05, 0.9`. You can use `is_new_plot=False` to show both plots on the same graph.

In [ ]:
eta, gamma = 0.05, 0.9

#
# YOUR CODE HERE
#

## Adagrad

Same for `adagrad(x, s)` with $\varepsilon = 10^{-6}$, and test it with `eta = 0.4` and `eta = 1.5`.

In [ ]:
#
# YOUR CODE HERE (hint: use x**a for coordinate-wise power a)
#

## RMSProp

Same for `rmsprop(x, s)`, and test it with `eta, gamma = 0.4, 0.9`.

In [ ]:
#
# YOUR CODE HERE
#

## Adam

Same for `adam(x, s)`, where the memory state will be a tuple `m, v, t = s` where `m` and `v` are the internal states, while `t` is the current iteration. Test it with `beta1, beta2, eta = 0.9, 0.99, 0.8`, and remember to initialize the internal state in the function `show_trace_2d`, here to `init_s=(0, 0, 0)`.

In [ ]:
#
# YOUR CODE HERE
#

## AMSGrad

Same for `amsgrad(x, s)`, where the memory state will be a tuple `m, v, v_bar = s` (see slides for the meaning of each value). Compare Adam and AMSGrad with `beta1, beta2, eta = 0.9, 0.99, 1`.

In [ ]:
#
# YOUR CODE HERE
#

# Stochastic optimization and schedulers

We will now see what happens when the gradient is noisy.

First, write a function `sgd(x, s)` that performs one step of stochastic gradient descent, by adding a noise of standard deviation $0.8$ to the gradient at each iteration using `get_gradient(f, x, noise_std=0.8)`. Compare the effect of different different step-sizes $\eta\in\{0.4,0.2,0.05\}$ on the convergence.

In [ ]:
#
# YOUR CODE HERE
#

A high learning rate is faster, but a small learning rate leads to better final performance.
Propose a modification of sgd that use behavior this to its advantage in an update rule named `sgd_with_scheduler`. Compare its behavior with standard sgd in the previous setting.

In [ ]:
#
# YOUR CODE HERE
#

We are now going to test sgd on a more realistic example: linear classification of a toy dataset. First, we generate a dataset with 2 classes, each being a distributed according to a Gaussian distribution.

In [ ]:
# Generating a dataset of two Gaussian classes and 100 samples
N = 100
v = torch.Tensor([1, 1]).unsqueeze(0)
Y = 2 * torch.randint(low=0, high=2, size=(N, 1), dtype=torch.float) - 1
X = 1 * torch.randn(N, 2) + Y @ v
plt.plot(X[Y.flatten() > 0,0], X[Y.flatten() > 0,1], 'o', color='red')
plt.plot(X[Y.flatten() < 0,0], X[Y.flatten() < 0,1], 'o', color='blue')

Now, write `mse_linear(theta)` that returns the MSE loss over the toy dataset for a simple linear model $y = \theta X$, as well as `mse_linear_mb(theta)` that returns the MSE loss over a random mini-batch of size `mini_batch_size`.

In [ ]:
#
# YOUR CODE HERE
# WARNING: theta should be of size (2,1)
#

mini_batch_size = 1
#
# YOUR CODE HERE
# WARNING: theta should be of size (2,1)
# HINT: use torch.randperm
#

Now, test a mini-batch version of Adam on this toy dataset, with `eta=0.1` and `mini_batch_size` in $\{1,5,10\}$. What do you see?

In [ ]:
#
# YOUR CODE HERE
#

# Pytorch optimizers and schedulers
Typical training loop, optimizers and schedulers in Pytorch

In [ ]:
# MINIMAL (AND INCOMPLETE) EXAMPLE
for epoch in range(20):
    for inputs, targets in dataloader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

In [ ]:
# MORE DETAILED AND COMPLETE EXAMPLE

### PARAMETERS
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
num_epochs = 100

### DATA, MODEL, LOSS, OPTIMIZER AND SCHEDULER
dataloader = ... # YOUR DATA
model = ... # YOUR MODEL
criterion = ... # YOUR LOSS FUNCTION
optimizer = ... # YOUR OPTIMIZER
scheduler = ... # YOUR SCHEDULER

### TRAINING LOOP
# Prepares the model for training (needed for some models)
model.train()

for epoch in range(num_epochs):
    # One training epoch over the whole dataset
    for inputs, targets in dataloader:
        # One mini-batch, put on the desired devide (cpu or gpu)
        inputs, targets = inputs.to(device), targets.to(device)

        # Reinitialize the gradients before any computation
        optimizer.zero_grad()

        # Computation of the model's output and loss on the mini-batch
        outputs = model(inputs)
        loss = criterion(outputs, targets)

        # Computation of the gradient on the mini-batch
        loss.backward()

        # One iteration of the optimizer and update of the step-size
        optimizer.step()

        # Then we can compute statistics and store loss values
        ...

    # Update of the step-size
    scheduler.step()
    print('Loss: {:.4f} Acc: {:.4f}'.format(..., ...))

In [ ]:
# Typical optimizers and schedulers
import torch.optim as optim

optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
optimizer = optim.Adam(model.parameters(), lr=0.0001)

scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer)
scheduler = optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.9)
scheduler = optim.lr_scheduler.LambdaLR(optimizer, lr_lambda=lambda t: 1/t**0.5)

In [ ]:
# If you want to implement your own optimizer
class SuperDuperOptimizer(optim.Optimizer):
    def __init__(self, params, lr):
        defaults = dict(lr=lr)
        super(SuperDuperOptimizer, self).__init__(params, defaults)

    def step():
        ...